Title: Task 6D **pandas vs SQL**<br> Name: **HARPREET SINGH** <br>Student Number: **223925166**<br> Email : **yashu.harpreet@gmail.com**

This task is about the use of **pandas python library** to come up with the results equivalent to those that correspond to **SQL
queries** perform on the given dataset<br>The dataset,**nycflights13** gives information about all 336,776 flights that departed
in 2013 from the three New York (in the US) airports (EWR, JFK, and LGA) to destinations in the United
States, Puerto Rico, and the American Virgin Islands<br>
nycflights13_flights.csv.gz – flights information,<br>
nycflights13_airlines.csv.gz – decodes two letter carrier codes,<br>
nycflights13_airports.csv.gz – airport data,<br>
nycflights13_planes.csv.gz – plane data,<br>
nycflights13_weather.csv.gz – hourly meteorological data for LGA, JFK, and EWR.

First, we import all the necessary libraries for performing the tasks

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tempfile, os.path
import sqlite3

The above libraries are imported successfully

Now we read the datasets using **pandas.read_csv** to import the data into respective datasets

In [2]:
airlines = pd.read_csv('nycflights13_airlines.csv.gz', compression='gzip',comment="#")
airports = pd.read_csv('nycflights13_airports.csv.gz', compression='gzip',comment="#")
flights = pd.read_csv('nycflights13_flights.csv.gz', compression='gzip',comment="#")
planes = pd.read_csv('nycflights13_planes.csv.gz', compression='gzip',comment="#",na_values=np.nan)
weather = pd.read_csv('nycflights13_weather.csv-1.gz', compression='gzip',comment="#")

All the datasets are read successfuly. Now we check the data in each dataset to have a glimpse of records present 

In [3]:
airlines.head(1)

,carrier,name
0,9E,Endeavor Air Inc.


In [4]:
airports.head(1)

,faa,name,lat,lon,alt,tz,dst,tzone
0,04G,Lansdowne Airport,41.130472,-80.619583,1044,-5,A,America/New_York


In [5]:
flights.head(1)

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00


In [6]:
planes.head(1)

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan


In [7]:
weather.head(1)

,origin,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
0,EWR,2013,1,1,0,37.04,21.92,53.97,230.0,10.35702,11.918651,0.0,1013.9,10.0,2013-01-01 01:00:00


All datasets are read and printed successfully

<br>

Now we create a **temporary database** by making a connection to the **sqlite** and load the datasets

In [8]:
import tempfile, os.path
dbfile = os.path.join(tempfile.mkdtemp(), "travel.db")
print(dbfile)

C:\Users\harpr\AppData\Local\Temp\tmpv41pzefr\travel.db


The travel.db database is successfully created in the above path **C:\Users\harpr\AppData\Local\Temp\tmpjb263djd\travel.db**

In [9]:
conn = sqlite3.connect(dbfile)
print(conn)

**sqlite3** connection is successfully made

Now all the datsets will be loaded into the database using **to_sql** function

In [10]:
airlines.to_sql("airlines", conn, index=False)
airports.to_sql("airports", conn, index=False)
flights.to_sql("flights", conn, index=False)
planes.to_sql("planes", conn, index=False)
weather.to_sql("weather", conn, index=False)

26130

<br>

Now we are going to **run sql queries and equivalent pandas code to get the same results**

In [11]:
# Sql query
task1_sql = pd.read_sql_query("""
SELECT DISTINCT engine FROM planes
""", conn)

The above **task1_sql sql query fetches the distinct engine values from the planes table**

In [12]:
task1_my = (
pd.DataFrame(planes['engine'].unique(),columns = ['engine'])
)

**task1_my** code uses the **unique() method on engine columns of planes dataframe to get the unique values**

In [13]:
pd.testing.assert_frame_equal(task1_sql, task1_my)

<br><br>

**task2_sql** sql query fetches **the distinct type an engine values from planes table**

In [14]:
task2_sql = pd.read_sql_query("""
SELECT DISTINCT type, engine FROM planes
""", conn)

In [15]:
task2_my = (
planes[['type', 'engine']].drop_duplicates().reset_index(drop=True)
)

**task2_my** code uses **drop_duplicates method on the planes dataset, which removes the duplicate rows from the dataset to get unique values**

In [16]:
pd.testing.assert_frame_equal(task2_sql, task2_my)

<br><br>

In [17]:
task3_sql = pd.read_sql_query("""
SELECT COUNT(*), engine FROM planes GROUP BY engine
""", conn)

**task3_sql sql counts the number of records for each engine**

In [18]:
task3_my = (
planes.groupby('engine').size().to_frame('COUNT(*)').
    reset_index()[['COUNT(*)','engine']]
)

**task3_my code groups the planes dataset by engine using groupby method , then counts the rows using size method**

In [19]:
pd.testing.assert_frame_equal(task3_sql, task3_my)

<br><br>

In [20]:
task4_sql = pd.read_sql_query("""
SELECT COUNT(*), engine, type FROM planes
GROUP BY engine, type
""", conn)

**task4_sql counts the number of records based on engine and type from the planes table**

In [21]:
task4_my = (
planes.groupby(['engine','type']).size().
    to_frame('COUNT(*)').reset_index()
    [['COUNT(*)','engine','type']]
)

**task4_my code uses groupby method on engine and type column to group the dataset first and the uses size method to count the respective rows**

In [22]:
pd.testing.assert_frame_equal(task4_sql, task4_my)

<br><br>

In [23]:
task5_sql = pd.read_sql_query("""
SELECT MIN(year), AVG(year), MAX(year), engine, manufacturer
FROM planes
GROUP BY engine, manufacturer
""", conn)

**task5_sql fetches the minimum, average ,maximum year values based on engine and manufacturer**

In [24]:
task5_my = (
planes.groupby(['engine', 'manufacturer'])[['year']].
    describe().droplevel(level=0,axis = 'columns').
    reset_index()[['engine', 'manufacturer', 'mean', 'min','max']].
    rename(columns = {"min":"MIN(year)",
                      "mean":"AVG(year)",
                      "max":"MAX(year)"})
    [['MIN(year)','AVG(year)', 'MAX(year)', 'engine', 'manufacturer']]
    
    # [['engine','manufacturer','min','mean','max']]
)

**task5_my code first groups the dataset using groupby method, calculating the statistics values using describe method and then extacting the minimum,average and maximum year values**

In [25]:
pd.testing.assert_frame_equal(task5_sql, task5_my)

<br><br>

In [26]:
task6_sql = pd.read_sql_query("""
SELECT * FROM planes WHERE speed IS NOT NULL
""", conn)

**task6_sql sql fetches all records where speed value exists**

In [27]:
task6_my = (
planes[planes['speed'].notnull()].reset_index(drop = True)
)

**task6_my code check not null values using  notnull method on speed column and then filters the planes dataset where notnull method retun true values**

In [28]:
pd.testing.assert_frame_equal(task6_sql, task6_my)

<br><br>

In [29]:
task7_sql = pd.read_sql_query("""
SELECT tailnum FROM planes
WHERE seats BETWEEN 150 AND 210 AND year >= 2011
""", conn)

**task7_sql sql fetches the tailnum from planes table where seat value lies between 150 and 210 and year is greater equal to 2011**

In [30]:
task7_my = (
planes.loc[(planes['year'] >= 2011) & ((planes['seats']>=150) & (planes['seats']<=210)),['tailnum']]
).reset_index(drop = True)

**task7_my code uses loc method which filters the data using year/seats column by applying the & logical operation**

In [31]:
pd.testing.assert_frame_equal(task7_sql, task7_my)

<br><br>

In [32]:
task8_sql = pd.read_sql_query("""
SELECT tailnum, manufacturer, seats FROM planes
WHERE manufacturer IN ("BOEING", "AIRBUS", "EMBRAER") AND seats>390
""", conn)

**task8_sql sql fetches the tailnum, manufacturer, seats from planes only for manufacturer belonging to "BOEING", "AIRBUS", "EMBRAER") and seats more than 390**

In [33]:
task8_my = (
planes.loc[(planes['manufacturer'].isin(["BOEING", "AIRBUS", "EMBRAER"])) & (planes['seats']>390),['tailnum','manufacturer','seats']].reset_index(drop = True)
)

**task8_my code uses isin method which return true values for the "BOEING", "AIRBUS", "EMBRAER" in manufacturer column as well as using & operation to seats column , then finally filters using loc method**

In [34]:
pd.testing.assert_frame_equal(task8_sql, task8_my)

<br><br>

In [35]:
task9_sql = pd.read_sql_query("""
SELECT DISTINCT year, seats FROM planes
WHERE year >= 2012 ORDER BY year ASC, seats DESC
""", conn)

**task9_sql sql fetches unique values where year greater than 2012 and fetches the results by ordering according to year and seats column**

In [36]:
task9_my = (
planes.loc[planes['year'] >= 2012]
    [['year', 'seats']].drop_duplicates().
    reset_index(drop = True).sort_values(by = ['year','seats']
                                         ,ascending = [True,False]).
    reset_index(drop = True)
)


**task9_my code filters the planes data first using loc method , then removing duplicate method to get unique values and then using sort_values function to get the results**

In [37]:
pd.testing.assert_frame_equal(task9_sql, task9_my)

<br><br>

In [38]:
task10_sql = pd.read_sql_query("""
SELECT DISTINCT year, seats FROM planes
WHERE year >= 2012 ORDER BY seats DESC, year ASC
""", conn)

**task10_sql sql fetches the unique year seats values where year is greater than 2012 and then ordering the results**

In [39]:
task10_my = (
planes.loc[planes['year'] >= 2012]
    [['year', 'seats']].drop_duplicates().
    reset_index(drop = True).sort_values(by = ['seats','year']
                                         ,ascending = [False,True]).
    reset_index(drop = True)
)

**task10_my code first filters the data according to year, then using drop_duplicates method to remove duplicated values from year, seat columns and then finally sorts the results based on seats and year column**

In [40]:
pd.testing.assert_frame_equal(task10_sql, task10_my)

<br><br>

In [41]:
task11_sql = pd.read_sql_query("""
SELECT manufacturer, COUNT(*) FROM planes
WHERE seats > 200 GROUP BY manufacturer
""", conn)

**task11_sql sql counts the number of records based on differnet manufacturers where seats are greater than 200**

In [42]:
task11_my = (
planes.loc[planes['seats'] > 200].groupby('manufacturer').size().to_frame("COUNT(*)").reset_index()
)

**task11_my code first filters the results using logical condition,then group the result using groupby method on manufacturer column and finally counting the records using size method**

In [43]:
pd.testing.assert_frame_equal(task11_sql, task11_my)

<br><br>

In [44]:
task12_sql = pd.read_sql_query("""
SELECT manufacturer, COUNT(*) FROM planes
GROUP BY manufacturer HAVING COUNT(*) > 10
""", conn)

**task12_sql sql counts the number of records for different manufacturers having records greater than 10**

In [45]:
task12_my = (
    planes.groupby('manufacturer').filter(lambda x: len(x) > 10)
    .groupby('manufacturer').size().to_frame(name = 'COUNT(*)').reset_index()
)

**task12_my code first groups the result using groupby method, filters the data using lambda function having records greater than 10, then again groups the result and finally counts the records using size method to count the number of records**

In [46]:
pd.testing.assert_frame_equal(task12_sql, task12_my)

<br><br>

In [47]:
task13_sql = pd.read_sql_query("""
SELECT manufacturer, COUNT(*) FROM planes
WHERE seats > 200 GROUP BY manufacturer HAVING COUNT(*) > 10
""", conn)

**task13_sql sql counts the number of records for different manufacturers which has more than 200 seats and totl records greater than 10**

In [48]:
task13_my = (
planes.loc[planes['seats'] > 200].groupby("manufacturer").size().to_frame("COUNT(*)").reset_index()
    [planes.loc[planes['seats'] > 200].groupby("manufacturer").size().to_frame("COUNT(*)").
     reset_index()["COUNT(*)"]>10].reset_index(drop = True)
)

**task13_my code first filters the data having seats greater than 200, groups the result on manufacturer using groupby methodand count the number of records that gives a dataframe.the resulting data frame is filtered again on counts ahving values greater than 10**

In [49]:
pd.testing.assert_frame_equal(task13_sql, task13_my)

<br><br>

In [50]:
task14_sql = pd.read_sql_query("""
SELECT manufacturer, COUNT(*) AS howmany
FROM planes
GROUP BY manufacturer
ORDER BY howmany DESC LIMIT 10
""", conn)

**task14_sql sql counts the number of records for different manufacturers and ordering the result in descending order and finally getting top 10 values**

In [51]:
task14_my = (
planes.groupby("manufacturer").size().to_frame("howmany").reset_index().
    sort_values(by = 'howmany',ascending = False).head(10).reset_index(drop = True)
)

**task14_my code first groups the result using groupby on manufacturer column, counting the records using size method,sorting the results using sort_values method and finally getting top 10 values using head method**

In [52]:
pd.testing.assert_frame_equal(task14_sql, task14_my)

<br><br>

In [53]:
task15_sql = pd.read_sql_query("""
SELECT
flights.*,
planes.year AS plane_year,
planes.speed AS plane_speed,
planes.seats AS plane_seats
FROM flights LEFT JOIN planes ON flights.tailnum=planes.tailnum
""", conn)

**task15_sql sql joins flights table and planes table using left join**

In [54]:
task15_my = (
pd.merge(flights, planes[['year','speed','seats','tailnum']], 
         left_on='tailnum',right_on='tailnum',
         how = 'left',suffixes=['_flights','_planes']).reset_index(drop = True).
    rename(columns = {"year_flights":"year",
                     "year_planes":"plane_year",
                     "speed":"plane_speed",
                     "seats":"plane_seats"})
)

**task15_my code joins flights and planes dataset using merge method on tailnum column key value** 

In [55]:
pd.testing.assert_frame_equal(task15_sql, task15_my)

<br><br>

In [56]:
task17_sql = pd.read_sql_query("""
SELECT
flights2.*,
atemp,
ahumid
FROM (
SELECT * FROM flights WHERE origin='EWR'
) AS flights2
LEFT JOIN (
SELECT
year, month, day,
AVG(temp) AS atemp,
AVG(humid) AS ahumid
FROM weather
WHERE origin='EWR'
GROUP BY year, month, day
) AS weather2
ON flights2.year=weather2.year
AND flights2.month=weather2.month
AND flights2.day=weather2.day
""", conn)

**task17_sql sql creates a temp table as flights2,using this table to have a left join with weather table  having average temp and average humid values for EWR origin,which uses  year,month and day key column for joining  and finally getting all columns from flight2 and average temp and average humid values**

In [57]:
task17_my = (
pd.merge(flights.loc[flights['origin'] == 'EWR'],
    weather[weather['origin'] == "EWR"].groupby(['year', 'month','day']).agg({'temp':'mean','humid':'mean'})
    .reset_index().rename(columns = {"temp":"atemp",
                                    "humid":"ahumid"}),
         how='left',on=['year','month','day'])
)

**task17_my code first filters the flights table for EWR origin, then filters weather table for EWR origin,grouping the result using groupby on year,month and day column, which further uses agg method to calculate mean of temp and humid columns. It then merges the two dataset using merge method to give the final result**

In [58]:
pd.testing.assert_frame_equal(task17_sql, task17_my)

<br>

In this report , we run the sql queries on the given dataset and after that we run the pandas code using the same logic used in sql queries and check if we are getting the same result.<br> For this first we load the dataset into temporary table using sqlite and to_sql function. we alse use various pandas functions for manipulating the data to obtain the similar results returned by sql queries